In [1]:
# Take-home assignment for Machine Learning Scientist position
# check if all necessary packages have been installed: pandas, sklearn, tensorflow, shap

Summary: <br>
    1. Assumptions：<br>
        day-to-day and fermenter-to-fermenter variablity don't affect the OD600. <br>
    2. Approach：<br>
        a. Data gathering and preprocessing:<br>
        Interpolated OD600 to generate more data for model training. Aligned the online parameters with the corresponding offline OD600 timestamps to build traning data and training sequences of length time_step = 10 for LSTMs model training for next step. Features are standardized to prevent dominance of one features. <br>
        b. Model Setup <br>
        LSTMs (Long Short-Term Memory networks) has been used as the basic model architecture since it excels at capturing temporal dependencies within time-series data. Built a univariate-output LSTM model to predict OD600 from multiple online features. Used mean squared error (MSE) as the loss function. Tried dropout (0.2–0.3) and varied the number of LSTM units (e.g., 50 → 20) to combat overfitting. Split the dataset into training and validation sets (e.g., 90%/10%) without shuffling (time-series respect). <br>
        c. Model Training and Hyperparameter Tuning <br>
        Ran multiple epochs (e.g., 20–50). Observed training loss vs. validation loss to diagnose overfitting. Adjusted dropout, the number of hidden units, and tried smaller or larger learning rates to improve generalization.<br>
        d. Prediction of Missing OD600 Data <br>
        After training, used the model to infer the OD600 time-series trendline for fermenters T3-080724, T6-080724, T7-120224, and T8-120224 (which were automatically found by comparing the missing exp_id between offline OD data file and online params file). <br>
        e. Feature Importance Analysis <br>
        SHAP (SHapley Additive exPlanations), a powerful method used to interpret the predictions of machine learning models, especially neural networks. Here, the GradientExplainer has been used to find the feature importance.<br>   
    3. Conclusion：<br>
    a. Model Predictablity:<br>
        The LSTM model learned to approximate biomass growth trends reasonably, but it showed signs of overfitting, evidenced by a large gap between training loss and validation loss. Nevertheless, within certain intervals, the model could capture OD600 trajectories with moderate fidelity. The predictability is constrained by OD600 generation with interpolation. The data makes it challenging to capture the real OD trend.  <br>
    b. Key Causal Features <br> 
        By applying SHAP, I observed that pH_control_buffer_total and feed_total onsistently showed higher attribution scores for OD600 prediction. These features collectively influenced how the model tracked biomass growth. <br>
    c. Practical Implications <br>
        The model’s ability to forecast biomass could assist in real-time monitoring and decision-making for fermentation processes, but additional data (more offline OD points) and refined regularization are recommended to improve robustness and reduce overfitting. <br>
    d. Additional questions (Is the assumption reasonable?) <br>
        The reason of overfitting is likely to come from the assumption which ignores the affects of fermenters and days。 Fermentation microbes may undergo mutations during their division and growth, leading to different batches exhibiting different OD trends. Euipment evolutions can also cause unexpected change to OD trends. My model has ability to distinguish 'exp_id'. To alter the model to quality the question, one way worth to try is to add categorical indicators as new features for day and dermenter, concatenate these with my usual time-series inputs before feeding them into the LSTM, and then analyze the learned embeddings to see how OD600 predictions shift across different days and fermenters. More advanced approach could be using a hierarchical or mixed-effects method if you want formal random effects and more structured interpretability.   
        
        

In [3]:
# Import the data preprocessing module
import data_preprocessing
import LSTM_Regression
import Predict_missing_data
import pandas as pd

In [4]:

# File paths to all data
online_params_file_1 = 'Time_series_online_params.csv'
online_params_file_2 = 'Timeseries_run_online_params_120224.csv'
od_data_file = 'Time_series_OD_data.csv'

# Get columns name for features and target
feature_cols = ['Agitation','DO', 'feed_flowrate', 'feed_total', 'pH', 'pH_control_buffer_flowrate', 'pH_control_buffer_total', 'product_inducer_flowrate', 'product_inducer_total']
target_col = ['OD600']

df_online_1 = data_preprocessing.load_online_params(online_params_file_1)
df_online_2 = data_preprocessing.load_online_params(online_params_file_2)
# Concatenate the DataFrames
df_online = pd.concat([df_online_1, df_online_2], axis=0).reset_index(drop=True)
df_od = data_preprocessing.load_od_data(od_data_file)

# Preprocess data to get standardized dataset with exp_id, time_hr, features, and target
preprocessed_data, scalar = data_preprocessing.preprocess_data(df_online, df_od, feature_cols, target_col)
# print(preprocessed_data)


C:\Users\10060\Downloads\Boston_Bioprocess\data_preprocessing.py:70: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  interpolated_group = numeric_group.interpolate(method='linear')
C:\Users\10060\Downloads\Boston_Bioprocess\data_preprocessing.py:70: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  interpolated_group = numeric_group.interpolate(method='linear')
C:\Users\10060\Downloads\Boston_Bioprocess\data_preprocessing.py:70: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  interpolated_group = numeric_group.interpolate(method='linear')
C:\Users\10060\Downloads\Boston_Bioprocess\data_preprocessing.py:70: FutureWarning: D

In [5]:
# Extract features and target
features = preprocessed_data[feature_cols].to_numpy()
target = preprocessed_data[target_col].to_numpy()

# Define the number of timesteps for sequences
time_steps = 10

# Create sequences for LSTM
X, y = LSTM_Regression.create_sequences_for_multiple_series(preprocessed_data, 'exp_id', feature_cols, target_col, time_steps)



In [6]:
# Train the LSTM model
Regression_model, mistory_model, mse, X_train, X_val = LSTM_Regression.LSTM_model(X,y,0.1)

Epoch 1/20


C:\Users\10060\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1670/1670 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 6039.7915 - val_loss: 489.4005
Epoch 2/20
1670/1670 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 918.1878 - val_loss: 624.2961
Epoch 3/20
1670/1670 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 844.2747 - val_loss: 700.8958
Epoch 4/20
1670/1670 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 817.4377 - val_loss: 649.6716
Epoch 5/20
1670/1670 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 738.6742 - val_loss: 838.0960
Epoch 6/20
1670/1670 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 689.4185 - val_loss: 1146.2966
Epoch 7/20
1670/1670 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 685.0648 - val_loss: 779.8785
Epoch 8/20
1670/1670 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 666.0085 - val_loss: 1424.4464
Epoch 9/20
1670/1670 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 637.8809 - val_loss: 1403.5656
Epoch 10/20
1670/1670 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 614.9761 - val_loss: 981.4994
Epoch 11/20
1670/1670 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 601.3454 - val_loss: 1

In [7]:
# Find the exp_id which have missing OD values
exp_id_x_missing = Predict_missing_data.find_exp_id_for_x_missing(df_online, df_od)
# print(exp_id_x_missing)

In [8]:
# Standardize the features for missing data prediction
dfs = []
for series_id in exp_id_x_missing:
    df_series = df_online[df_online['exp_id'] == series_id]
    standardize_df_series = data_preprocessing.standardize_data_only_features(df_series, feature_cols)
    dfs.append(standardize_df_series)
Standardized_missing_online = pd.concat(dfs, ignore_index=True)

In [9]:
# Generate input sequences for the new standardized feature data
feature_cols = ['Agitation','DO', 'feed_flowrate', 'feed_total', 'pH', 'pH_control_buffer_flowrate', 'pH_control_buffer_total', 'product_inducer_flowrate', 'product_inducer_total']
target_col = ['OD600']
X_missing, seq_metadata = LSTM_Regression.create_sequences_for_multiple_series_inference(
    df=Standardized_missing_online,
    id_col="exp_id",
    feature_cols=feature_cols,
    time_steps=time_steps
)

In [10]:
# Get the predicted OD
predictions = Predict_missing_data.predict_and_postprocess(Regression_model, X_missing)


620/620 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [11]:
# Convert predictions and metadata to a DataFrame for further use
predictions_df = pd.DataFrame(seq_metadata)
predictions_df['predicted_OD600'] = predictions.flatten()
# print(predictions_df)

In [12]:
# Find the feature importance
feature_importance = Predict_missing_data.SHAP_feature_importance(X_train, X_val, Regression_model)

C:\Users\10060\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=['Tensor(shape=(10, 10, 9))']
  warnings.warn(msg)
C:\Users\10060\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=['Tensor(shape=(50, 10, 9))']
  warnings.warn(msg)


In [13]:
# To get feature-only importance (not considering time series importance):
import numpy as np

shap_values_agg_time = np.mean(np.abs(feature_importance), axis=1)    # remove time
# shap_values_agg_time.shape -> (n_samples, n_features)

feature_importance_final = np.mean(shap_values_agg_time, axis=0)     # remove samples
# feature_importance.shape -> (n_features,)

for i, score in enumerate(feature_importance_final):
    print(f"Feature {i}: {score}")

Feature 0: [1.73955991]
Feature 1: [0.33163232]
Feature 2: [0.43152321]
Feature 3: [1.93808462]
Feature 4: [0.35149593]
Feature 5: [0.2316468]
Feature 6: [4.27552404]
Feature 7: [1.53940063]
Feature 8: [0.66083893]


In [15]:
# Extract the values from the lists and combine them with their indices
feature_importances = [(i, score[0]) for i, score in enumerate(feature_importance_final)]

# Sort the features by their importance scores in descending order
sorted_features = sorted(feature_importances, key=lambda x: x[1], reverse=True)

# Get the top two features with the highest importance values
top_two_features = sorted_features[:2]

# Print the two most important features
for i, score in top_two_features:
    print(feature_cols[i], " with ", score)

pH_control_buffer_total  with  4.275524036139251
feed_total  with  1.9380846180065419
